### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Dutch
Lang Id = 6
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,we zijn er dat was het dat was,473,476,nou rik we zijn er dat was het dat was hem,1ZuKeTOZ7IU,https://www.youtube.com/watch?v=1ZuKeTOZ7IU&t=...
1,niet voor het ik maar voor het,4037,4039,dus niet voor het ik maar voor het wij,ufQenq834F4,https://www.youtube.com/watch?v=ufQenq834F4&t=...
2,maar niet als je het niet,793,795,maar niet als je het niet leuk vond,IOMexym5byo,https://www.youtube.com/watch?v=IOMexym5byo&t=...
3,maar niet als je het niet,832,835,maar niet als je het niet leuk vond,2RDPz03xJMg,https://www.youtube.com/watch?v=2RDPz03xJMg&t=...
4,maar niet als je het niet,734,736,maar niet als je het niet leuk vond,3r234qU24b4,https://www.youtube.com/watch?v=3r234qU24b4&t=...
...,...,...,...,...,...,...
195,en als we een,148,155,en als we een t,27lgde1JmnI,https://www.youtube.com/watch?v=27lgde1JmnI&t=...
196,je wat het is,47,48,weet je wat het is,E802g11Qy_Y,https://www.youtube.com/watch?v=E802g11Qy_Y&t=47s
197,ik heb er een,411,412,ja ik heb er een,fFryfrZY8AM,https://www.youtube.com/watch?v=fFryfrZY8AM&t=...
198,en wat is het,257,259,en wat is het nu,BhuUS2JmcpA,https://www.youtube.com/watch?v=BhuUS2JmcpA&t=...


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,dan toch mee omdat ze weet dat de vader anders...,264,270,dan toch mee omdat ze weet dat de vader anders...,knaPebpdfcI,https://www.youtube.com/watch?v=knaPebpdfcI&t=...
1,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...
2,daar is nog wel iets meer over te zeggen maar ...,703,708,daar is nog wel iets meer over te zeggen maar ...,gQPOBsiqB_o,https://www.youtube.com/watch?v=gQPOBsiqB_o&t=...
3,en het komt ook omdat je heel goed moet weten ...,203,210,en het komt ook omdat je heel goed moet weten ...,c5DtYHHKMK0,https://www.youtube.com/watch?v=c5DtYHHKMK0&t=...
4,en toen dacht ik van nou dit moet ik hebben di...,146,155,en toen dacht ik van nou dit moet ik hebben di...,9crP_C_ZvB8,https://www.youtube.com/watch?v=9crP_C_ZvB8&t=...
...,...,...,...,...,...,...
195,hier niet alleen er zijn hier heel veel mensen...,249,252,en je traint hier niet alleen er zijn hier hee...,7b5MnOGVhzw,https://www.youtube.com/watch?v=7b5MnOGVhzw&t=...
196,en die gaan mij laten zien wat hier allemaal t...,343,345,en die gaan mij laten zien wat hier allemaal t...,QeXglirqQpc,https://www.youtube.com/watch?v=QeXglirqQpc&t=...
197,over wat wij vinden en van ja nee maar niemand...,4632,4637,over wat wij vinden en van ja nee maar niemand...,EmxRrMXp3UY,https://www.youtube.com/watch?v=EmxRrMXp3UY&t=...
198,die heeft daar echt heel veel werk voor gedaan...,150,155,in belgi die heeft daar echt heel veel werk vo...,9xycq3MyIQE,https://www.youtube.com/watch?v=9xycq3MyIQE&t=...


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik moet het ook gewoon zeggen anders gaat iema...,730,736,ik moet het ook gewoon zeggen anders gaat iema...,LemeIsc8Iy0,https://www.youtube.com/watch?v=LemeIsc8Iy0&t=...
1,dus wanneer je iemand anders ziet eten dat je ...,365,372,dus wanneer je iemand anders ziet eten dat je ...,jSq4KqjykvM,https://www.youtube.com/watch?v=jSq4KqjykvM&t=...
2,als je gewoon mensen bij elkaar aan tafel zet ...,295,304,als je gewoon mensen bij elkaar aan tafel zet ...,7qygIdY0k8Y,https://www.youtube.com/watch?v=7qygIdY0k8Y&t=...
3,nou het valt me niet tegen moet ik zeggen want...,98,104,nou het valt me niet tegen moet ik zeggen want...,URyrZjK46pQ,https://www.youtube.com/watch?v=URyrZjK46pQ&t=98s
4,laten we nou eens kijken wat er gebeurt als er...,576,584,laten we nou eens kijken wat er gebeurt als er...,oewqQ6RkNT8,https://www.youtube.com/watch?v=oewqQ6RkNT8&t=...
...,...,...,...,...,...,...
195,maar wilt op meer echt mensen gaan helpen dus ...,2746,2752,maar wilt op meer echt mensen gaan helpen dus ...,eUmaumYRtOE,https://www.youtube.com/watch?v=eUmaumYRtOE&t=...
196,daar hebben we eerder wel een keer onderzoek g...,1332,1337,daar hebben we eerder wel een keer onderzoek g...,CQ5B3DDNDwI,https://www.youtube.com/watch?v=CQ5B3DDNDwI&t=...
197,ze gaan we gewoon niet altijd begrijpen laat m...,4857,4862,van keek ze gaan we gewoon niet altijd begrijp...,eUmaumYRtOE,https://www.youtube.com/watch?v=eUmaumYRtOE&t=...
198,waarschijnlijk zijn dit wel een beetje dezelfd...,167,181,waarschijnlijk zijn dit wel een beetje dezelfd...,XwM8xs5ZbMg,https://www.youtube.com/watch?v=XwM8xs5ZbMg&t=...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,hier slapen alle dieren in rust en wij gaan de...,1251,1273,hier slapen alle dieren in rust en wij gaan de...,iJb2QNlOkpY,https://www.youtube.com/watch?v=iJb2QNlOkpY&t=...
1,nou ik ben in ieder geval heel blij mee en ik ...,1478,1487,nou ik ben in ieder geval heel blij mee en ik ...,4OE321XI9ko,https://www.youtube.com/watch?v=4OE321XI9ko&t=...
2,maar het is wel belangrijk als jullie groeien ...,547,557,maar het is wel belangrijk als jullie groeien ...,k1vwT6_PBi8,https://www.youtube.com/watch?v=k1vwT6_PBi8&t=...
3,maar veel verder kom ik niet want er is in een...,142,152,maar veel verder kom ik niet want er is in een...,jY7Wfl92qis,https://www.youtube.com/watch?v=jY7Wfl92qis&t=...
4,niet te kijken naar het gedrag van de mensen z...,27,35,daar stonden wij een keertje niet te kijken na...,YQh4Xulq0Fo,https://www.youtube.com/watch?v=YQh4Xulq0Fo&t=27s
...,...,...,...,...,...,...
195,op die manier konden meerdere personen met hun...,44,49,op die manier konden meerdere personen met hun...,CH1fzbfKwjk,https://www.youtube.com/watch?v=CH1fzbfKwjk&t=44s
196,wat we alleen wel hebben besloten uw kamer vra...,13115,13122,wat we alleen wel hebben besloten uw kamer vra...,QAjt9liE_a4,https://www.youtube.com/watch?v=QAjt9liE_a4&t=...
197,gelukkig nog steeds hoop daarom hebben we bewu...,1913,1920,gelukkig nog steeds hoop daarom hebben we bewu...,wehip3eUEi4,https://www.youtube.com/watch?v=wehip3eUEi4&t=...
198,van jouw leeftijd die ook gewoon te maken we m...,2706,2712,van jouw leeftijd die ook gewoon te maken we m...,Co4vE_sSnZM,https://www.youtube.com/watch?v=Co4vE_sSnZM&t=...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,kijk bijvoorbeeld naar een gebaar als wonen of...,234,244,kijk bijvoorbeeld naar een gebaar als wonen of...,oewqQ6RkNT8,https://www.youtube.com/watch?v=oewqQ6RkNT8&t=...
1,super leuk nu zie je het nog niet heel goed ma...,756,764,super leuk nu zie je het nog niet heel goed ma...,Z0czr5G9r0s,https://www.youtube.com/watch?v=Z0czr5G9r0s&t=...
2,maar veel verder kom ik niet want er is in een...,142,152,maar veel verder kom ik niet want er is in een...,jY7Wfl92qis,https://www.youtube.com/watch?v=jY7Wfl92qis&t=...
3,en ja dat is wel gewoon iets wat je toch ondan...,229,237,en ja dat is wel gewoon iets wat je toch ondan...,5S5QX4ZK3Q0,https://www.youtube.com/watch?v=5S5QX4ZK3Q0&t=...
4,ik kan me nu ook wel een beetje voorstellen ho...,166,174,ik kan me nu ook wel een beetje voorstellen ho...,c3kFCnzQdHs,https://www.youtube.com/watch?v=c3kFCnzQdHs&t=...
...,...,...,...,...,...,...
195,echt heel knap goed we gaan weer werd gevuld j...,735,742,echt heel knap goed we gaan weer werd gevuld j...,acVRitTW2fU,https://www.youtube.com/watch?v=acVRitTW2fU&t=...
196,misschien minder goede keuzes maken waardoor z...,1098,1106,misschien minder goede keuzes maken waardoor z...,3TgwrjeZLR0,https://www.youtube.com/watch?v=3TgwrjeZLR0&t=...
197,ook volwassenen die er voor getekend hebben da...,1453,1458,ook volwassenen die er voor getekend hebben da...,ogD6KUT7xrg,https://www.youtube.com/watch?v=ogD6KUT7xrg&t=...
198,daar komt heel veel druk op ons af wij moeten ...,2343,2349,daar komt heel veel druk op ons af wij moeten ...,25udztDPW_k,https://www.youtube.com/watch?v=25udztDPW_k&t=...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,dus als het gaat om de slecht nieuws gesprekke...,1110,1121,dus als het gaat om de slecht nieuws gesprekke...,QPALQDtz7so,https://www.youtube.com/watch?v=QPALQDtz7so&t=...
1,op moment van bedenken is het blijkbaar niet z...,333,347,op moment van bedenken is het blijkbaar niet z...,xKhL2kGvszM,https://www.youtube.com/watch?v=xKhL2kGvszM&t=...
2,en dan kun je wel zeggen daar heb ik eigenlijk...,1876,1887,en dan kun je wel zeggen daar heb ik eigenlijk...,37RqLFakmo0,https://www.youtube.com/watch?v=37RqLFakmo0&t=...
3,in ieder geval hartstikke bedankt ik denk dat ...,611,621,in ieder geval hartstikke bedankt ik denk dat ...,yD5aDc5N5qc,https://www.youtube.com/watch?v=yD5aDc5N5qc&t=...
4,wij hadden op dinsdag contact die dag daarna z...,386,394,wij hadden op dinsdag contact die dag daarna z...,LemeIsc8Iy0,https://www.youtube.com/watch?v=LemeIsc8Iy0&t=...
...,...,...,...,...,...,...
195,want onze wereld is niet gebouwd opdat ze er f...,2857,2865,want onze wereld is niet gebouwd opdat ze er f...,oboSnBnZxkA,https://www.youtube.com/watch?v=oboSnBnZxkA&t=...
196,die discussie was wat ons betreft nogal teleur...,16019,16027,die discussie was wat ons betreft nogal teleur...,k1up2KLkuaM,https://www.youtube.com/watch?v=k1up2KLkuaM&t=...
197,dus in meer levens man ben geweest om me daaro...,184,192,dus in meer levens man ben geweest om me daaro...,kJXfhcHUC3Q,https://www.youtube.com/watch?v=kJXfhcHUC3Q&t=...
198,hij heeft zelfs zn navelstreng er nog aanzitte...,1539,1545,hij heeft zelfs zn navelstreng er nog aanzitte...,hkYSfO501h0,https://www.youtube.com/watch?v=hkYSfO501h0&t=...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,dus als het gaat om de slecht nieuws gesprekke...,1110,1121,dus als het gaat om de slecht nieuws gesprekke...,QPALQDtz7so,https://www.youtube.com/watch?v=QPALQDtz7so&t=...
1,op moment van bedenken is het blijkbaar niet z...,333,347,op moment van bedenken is het blijkbaar niet z...,xKhL2kGvszM,https://www.youtube.com/watch?v=xKhL2kGvszM&t=...
2,het is inderdaad mogelijk om bij elkaar te bli...,261,270,het is inderdaad mogelijk om bij elkaar te bli...,7D5ZiQsUy48,https://www.youtube.com/watch?v=7D5ZiQsUy48&t=...
3,en afhankelijk van waar je woont in nederland ...,1103,1112,en afhankelijk van waar je woont in nederland ...,gQPOBsiqB_o,https://www.youtube.com/watch?v=gQPOBsiqB_o&t=...
4,en dan kun je wel zeggen daar heb ik eigenlijk...,1876,1887,en dan kun je wel zeggen daar heb ik eigenlijk...,37RqLFakmo0,https://www.youtube.com/watch?v=37RqLFakmo0&t=...
...,...,...,...,...,...,...
195,er niet twee duidelijke groepjes ontstaan veel...,316,325,er niet twee duidelijke groepjes ontstaan veel...,Pur8mxCAHeE,https://www.youtube.com/watch?v=Pur8mxCAHeE&t=...
196,nee denk niet deze man wel want hij kan nameli...,146,152,nee denk niet deze man wel want hij kan nameli...,zTvZsOzut_s,https://www.youtube.com/watch?v=zTvZsOzut_s&t=...
197,maar die koptelefoon waardoor jij misschien we...,171,177,maar die koptelefoon waardoor jij misschien we...,OSmw7CwaCQQ,https://www.youtube.com/watch?v=OSmw7CwaCQQ&t=...
198,terwijl heel veel organisaties nog werken met ...,701,706,terwijl heel veel organisaties nog werken met ...,zl1IPpX3sok,https://www.youtube.com/watch?v=zl1IPpX3sok&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1674725723
  nanos: 314817000
}